In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [27]:
def calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100

In [16]:
team_map = {
        'Wizards' : 'Washington Wizards',
        'Celtics' : 'Boston Celtics',
        'Pelicans' : 'New Orleans Pelicans',
        'Knicks' : 'New York Knicks',
        'Pistons' : 'Detroit Pistons',
        'Magic' : 'Orlando Magic',
        '76ers' : 'Philadelphia 76ers',
        'Hawks' : 'Atlanta Hawks',
        'Pacers' : 'Indiana Pacers',
        'Raptors' : 'Toronto Raptors',
        'Grizzlies' : 'Memphis Grizzlies',
        'Heat' : 'Miami Heat',
        'Bulls' : 'Chicago Bulls',
        'Jazz' : 'Utah Jazz',
        'Bucks' : 'Milwaukee Bucks',
        'Spurs' : 'San Antonio Spurs',
        'Warriors' : 'Golden State Warriors',
        'Thunder' : 'Oklahoma City Thunder',
        'Timberwolves' : 'Minnesota Timberwolves',
        'Nuggets' : 'Denver Nuggets',
        'Suns' : 'Phoenix Suns',
        'Cavaliers' : 'Cleveland Cavaliers',
        'Mavericks' : 'Dallas Mavericks',
        'Kings' : 'Sacramento Kings',
        'Hornets' : 'Charlotte Hornets',
        'Trail Blazers' : 'Portland Trailblazers',
        'Nets' : 'Brooklyn Nets',
        'Lakers' : 'Los Angeles Lakers',
        'Clippers' : 'Los Angeles Clippers',
        'Rockets' : 'Houston Rockets'
    }

In [3]:
# Instantiating WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/nba/odds')

# Navigating to ML
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')

# Getting table from HTML
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Get LATEST driver version for 98.0.4758
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.48/chromedriver_mac64.zip
Driver has been saved in cache [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.48]


In [29]:
odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():
    
    # Retreiving teams
    teams = {}
    for key in team_map.keys():
        if row.Scheduled.find(key) != -1:
            teams[row.Scheduled.find(key)] = key
    keys = []
    for key in teams.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams[keys[0]]
        away_team = teams[keys[1]]
    else:
        home_team = teams[keys[1]]
        away_team = teams[keys[0]]
    
    # Retreiving odds
    ml_string = row['Unnamed: 4']
    if len(ml_string) == 12:
        ml_string = ml_string.replace('ML', '')
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    if len(ml_string) == 13:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])

# Adding a message if games are passed over for same reason
if len(odds_df) != len(odds):
    print("At least one game not included in odds df")

In [19]:
odds

,Scheduled,Open,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,6:00 PM 501TimberwolvesTimberwolves502PistonsP...,-295+240,-290ML+230ML,-295ML+240ML,-295ML+240ML,-300ML+240ML,N/AN/A,-300ML+240ML,-300ML+240ML,-295ML+240ML,-310ML+250ML,-295ML+240ML
1,6:30 PM 505SunsSuns506HawksHawks,-210+176,-220ML+180ML,-220ML+180ML,-210ML+176ML,-227ML+185ML,N/AN/A,-225ML+180ML,-227ML+185ML,-220ML+180ML,-230ML+190ML,-220ML+180ML
2,6:30 PM 553BullsBulls554RaptorsRaptors,-240+190,+130ML-150ML,+135ML-162ML,+138ML-164ML,+135ML-160ML,N/AN/A,+135ML-160ML,+135ML-160ML,+135ML-162ML,+140ML-160ML,+135ML-162ML
3,7:30 PM 511HeatHeat512SpursSpurs,+118-145,-190ML+160ML,-180ML+148ML,-162ML+136ML,-180ML+150ML,+200ML-275ML,-190ML+155ML,-180ML+150ML,-195ML+160ML,-200ML+170ML,-195ML+160ML
4,9:00 PM 509KingsKings510WarriorsWarriors,+700-1100,+600ML-900ML,+600ML-910ML,+700ML-1100ML,+650ML-1000ML,N/AN/A,+600ML-900ML,+650ML-1000ML,+650ML-1115ML,+650ML-1000ML,+650ML-1115ML
5,9:00 PM 507LakersLakers508ClippersClippers,+118-138,+125ML-145ML,+120ML-143ML,+118ML-138ML,+125ML-145ML,N/AN/A,+115ML-140ML,+125ML-145ML,+120ML-143ML,+130ML-150ML,+120ML-143ML


In [ ]:

odds_df['Home_Prob'] = odds_df.Home_Odds.apply(calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])